<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np

RANDOM_STATE = 12345

In [2]:
data = pd.read_csv("/datasets/Churn.csv")

In [3]:
display(data.head(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [4]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

Первое, что бросается в глаза, что названия столбцов не в змеином формате. Исправим:

In [5]:
new_columns_name = {"RowNumber":"row_number","CustomerId":"customer_id","Surname":"surname",\
                    "CreditScore":"credit_score","Geography":"geography","Gender":"gender",\
                    "Age":"age","Tenure":"tenure","Balance":"balance",\
                    "NumOfProducts":"number_of_products","HasCrCard":"credit_card","IsActiveMember":"is_active",\
                    "EstimatedSalary":"estimated_salary","Exited":"exited"}
data.rename(columns = new_columns_name, inplace = True)

Так же, значение "tenture" имеет пропуски, а замена на среднее или медианное значение может плохо повлиять на будущее обучение, поэтому лучшем решением будет удалить строки:

In [6]:
data = data[~data["tenure"].isna()] 

Фамилия, айди и номер строки нам не нужны, поэтому их можно удалить:

In [7]:
data = data.drop(["surname","customer_id","row_number"], axis = 1)

Разделим данные на признаки (features) и на целевой признак (target):

In [8]:
features = data.drop(["exited"], axis = 1)
target = data["exited"]

Для создания модели и дальнейшей ее валидации, нам понадобится разделить данные на три группы: тренировочная, валидационная и тестировочная. Так как данных очень мало, то они будут разделены в соотношении 80%, 10% и 10%.

In [9]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.2, random_state=RANDOM_STATE, stratify = target)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=RANDOM_STATE, stratify = target_valid_test)

Далее, стандартизируем данные, чтобы у всех признаков была одинаковая важность:

In [10]:
numeric = ["credit_score","age","tenure","balance","number_of_products","estimated_salary"]

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_train = features_train.reset_index(drop=True)

features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid = features_valid.reset_index(drop=True)

features_test[numeric] = scaler.transform(features_test[numeric])
features_test = features_test.reset_index(drop=True)

/tmp/ipykernel_521/2455036538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/tmp/ipykernel_521/2455036538.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Далее, приведем классификационные признаки в формат OHE:

In [11]:
not_numeric = ["geography","gender"]

In [12]:
features_train_ohe = features_train[not_numeric]
features_valid_ohe = features_valid[not_numeric]
features_test_ohe = features_test[not_numeric]
features_train = features_train.drop(not_numeric, axis = 1)
features_valid = features_valid.drop(not_numeric, axis = 1 )
features_test = features_test.drop(not_numeric, axis = 1 )

In [13]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(features_train_ohe)

features_train_ohe =  pd.DataFrame(ohe.transform(features_train_ohe))
features_train_ohe = features_train_ohe.reset_index(drop=True)

features_valid_ohe =  pd.DataFrame(ohe.transform(features_valid_ohe))
features_valid_ohe = features_valid_ohe.reset_index(drop=True)

features_test_ohe = pd.DataFrame(ohe.transform(features_test_ohe))
features_test_ohe = features_test_ohe.reset_index(drop=True)

In [14]:
features_train = pd.concat([features_train_ohe,features_train], axis = 1)
features_valid = pd.concat([features_valid_ohe,features_valid], axis = 1)
features_test = pd.concat([features_test_ohe,features_test], axis = 1)

Данные готовы для первых проб в постройки модели. 

## Исследование задачи

Сначала, попробуем разные модели без балансировки и без подбора гиперпараметров:

In [15]:
original_result = pd.DataFrame(columns = ["F1","AUC ROC"])

**LogisticRegression**

In [16]:
model = LogisticRegression(random_state=RANDOM_STATE)
model.fit(features_train, target_train)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
original_result.loc['LR'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.7997799779977998
Precision: 0.5263157894736842
Recall: 0.16216216216216217
F1 Score: 0.24793388429752064
AUC ROC: 0.7455129162311482


**RandomForestClassifier**

In [17]:
model = RandomForestClassifier(random_state=RANDOM_STATE)
model.fit(features_train, target_train)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
original_result.loc['RFC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.8536853685368537
Precision: 0.7321428571428571
Recall: 0.44324324324324327
F1 Score: 0.5521885521885521
AUC ROC: 0.8396819471405107


**DecisionTreeClassifier**

In [18]:
model = DecisionTreeClassifier(random_state=RANDOM_STATE)
model.fit(features_train, target_train)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
original_result.loc['DTC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.7777777777777778
Precision: 0.4564102564102564
Recall: 0.4810810810810811
F1 Score: 0.46842105263157896
AUC ROC: 0.6673361206510378


Вывод:
Лучше всего себя показала модель RandomForestClassifier, со значением F1 = 0.552, что близко к цели, но требует доработки как данных, так и самой модели.

## Борьба с дисбалансом

In [19]:
print("Количество целевого признака класса 0:",target_train[target_train==0].count())
print("Количество целевого признака класса 1:",target_train[target_train==1].count())

Количество целевого признака класса 0: 5789
Количество целевого признака класса 1: 1483


Как можно наблюдать, соотношение целевого признака класса 0 и целевого признака класса 1 плохое. Намного больше ситуаций с целевым признаком 0. Следственно, нужно решить этот дисбаланс. Проверим четыре способа: Upsampling, Class_Weight, Downsampling и Threshold. В конце, выберем лучший метод. Результаты будем сравнивать по F1-мере и AUR ROC значению. 

**Upsampling**

In [20]:
print("Количество целевого признака класса 0:",target_train[target_train==0].count())
print("Количество целевого признака класса 1:",target_train[target_train==1].count())
print("Соотношение классов:",(target_train[target_train==0].count()/target_train[target_train==1].count()))

Количество целевого признака класса 0: 5789
Количество целевого признака класса 1: 1483
Соотношение классов: 3.9035738368172623


Чтобы достичь баланса классов, нам нужно иметь соотношение классов 1:1, для этого нужно увеличить количество целевых признаков класса 1 в четыре раза (ближайшее целое число). Сделать это можно за счет функции upsample_shuffle:

In [21]:
def upsample_shuffle(features,target,repeat):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_upsampled = pd.concat([features_zeros]+[features_ones]*repeat)
    target_upsampled = pd.concat([target_zeros]+[target_ones]*repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled)
    
    return features_upsampled, target_upsampled

In [22]:
features_train_upsampled, target_train_upsampled = upsample_shuffle(features_train.reset_index(drop=True),target_train.reset_index(drop=True),4)

Проверяем:

In [23]:
print("Количество целевого признака класса 0:",target_train_upsampled[target_train_upsampled==0].count())
print("Количество целевого признака класса 1:",target_train_upsampled[target_train_upsampled==1].count())
print("Соотношение классов:",(target_train_upsampled[target_train_upsampled==0].count()/target_train_upsampled[target_train_upsampled==1].count()))

Количество целевого признака класса 0: 5789
Количество целевого признака класса 1: 5932
Соотношение классов: 0.9758934592043156


Проверим какая модель лучше всего работает с улучшенными данными. Лучшую модель будет дорабатывать гиперпараметрами. 

In [24]:
upsampling_result = pd.DataFrame(columns = ["F1","AUC ROC"])

In [25]:
model = LogisticRegression(random_state=RANDOM_STATE)
model.fit(features_train_upsampled,target_train_upsampled)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
upsampling_result.loc['LR'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.6985698569856986
Precision: 0.3687315634218289
Recall: 0.6756756756756757
F1 Score: 0.47709923664122134
AUC ROC: 0.7496490966104224


In [26]:
model = RandomForestClassifier(random_state=RANDOM_STATE)
model.fit(features_train_upsampled,target_train_upsampled)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
upsampling_result.loc['RFC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.847084708470847
Precision: 0.6597222222222222
Recall: 0.5135135135135135
F1 Score: 0.5775075987841946
AUC ROC: 0.8413431387188294


In [27]:
model = DecisionTreeClassifier(random_state=RANDOM_STATE)
model.fit(features_train_upsampled,target_train_upsampled)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
upsampling_result.loc['DTC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.7821782178217822
Precision: 0.46368715083798884
Recall: 0.4486486486486487
F1 Score: 0.4560439560439561
AUC ROC: 0.658025981782888


**Weight_class**

Добавим во все модели гиперпараметр - class_weight = "balanced"

In [28]:
weight_class_result = pd.DataFrame(columns = ["F1","AUC ROC"])

In [29]:
model = LogisticRegression(random_state=RANDOM_STATE, class_weight = "balanced")
model.fit(features_train,target_train)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
weight_class_result.loc['LR'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.6996699669966997
Precision: 0.3674698795180723
Recall: 0.6594594594594595
F1 Score: 0.47195357833655704
AUC ROC: 0.74968642675825


In [30]:
model = RandomForestClassifier(random_state=RANDOM_STATE, class_weight = "balanced")
model.fit(features_train,target_train)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
weight_class_result.loc['RFC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.8514851485148515
Precision: 0.7358490566037735
Recall: 0.42162162162162165
F1 Score: 0.5360824742268042
AUC ROC: 0.845005226220696


In [31]:
model = DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight = "balanced")
model.fit(features_train,target_train)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
weight_class_result.loc['DTC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.7777777777777778
Precision: 0.455026455026455
Recall: 0.4648648648648649
F1 Score: 0.4598930481283422
AUC ROC: 0.6612998357473496


**Downsampling**

In [32]:
print("Количество целевого признака класса 0:",target_train[target_train==0].count())
print("Количество целевого признака класса 1:",target_train[target_train==1].count())
print("Соотношение классов:",(target_train[target_train==1].count()/target_train[target_train==0].count()))

Количество целевого признака класса 0: 5789
Количество целевого признака класса 1: 1483
Соотношение классов: 0.2561755052686129


Чтобы достичь баланса классов, нам нужно иметь соотношение классов 1:1, для этого нужно уменьшить количество целевых признаков класса 0 в четыре раза (1/0.25). Сделать это можно за счет функции downsample_shuffle:

In [33]:
def downsample_shuffle(features,target,fraction):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled)
    
    return features_downsampled, target_downsampled

In [34]:
features_train_downsampled, target_train_downsampled = downsample_shuffle(features_train.reset_index(drop=True), target_train.reset_index(drop=True), 0.25)

In [35]:
print("Количество целевого признака класса 0:",target_train_downsampled[target_train_downsampled==0].count())
print("Количество целевого признака класса 1:",target_train_downsampled[target_train_downsampled==1].count())
print("Соотношение классов:",(target_train_downsampled[target_train_downsampled==1].count()/target_train_downsampled[target_train_downsampled==0].count()))

Количество целевого признака класса 0: 1447
Количество целевого признака класса 1: 1483
Соотношение классов: 1.0248790601243953


In [36]:
downsampling_result = pd.DataFrame(columns = ["F1","AUC ROC"])

In [37]:
model = LogisticRegression(random_state=RANDOM_STATE)
model.fit(features_train_downsampled,target_train_downsampled)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
downsampling_result.loc['LR'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.6853685368536854
Precision: 0.3561253561253561
Recall: 0.6756756756756757
F1 Score: 0.4664179104477612
AUC ROC: 0.7494325817530237


In [38]:
model = RandomForestClassifier(random_state=RANDOM_STATE)
model.fit(features_train_downsampled,target_train_downsampled)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
downsampling_result.loc['RFC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.7667766776677668
Precision: 0.45484949832775917
Recall: 0.7351351351351352
F1 Score: 0.5619834710743802
AUC ROC: 0.8395102284605047


In [39]:
model = DecisionTreeClassifier(random_state=RANDOM_STATE)
model.fit(features_train_downsampled,target_train_downsampled)
predicted = model.predict(features_valid)
print("Accurasy:", model.score(features_valid,target_valid))
print("Precision:", precision_score(target_valid,predicted))
print("Recall:", recall_score(target_valid,predicted))
print("F1 Score:", f1_score(target_valid,predicted))
probabilities_one_valid = model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))
downsampling_result.loc['DTC'] = pd.Series({'F1':f1_score(target_valid,predicted), 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

Accurasy: 0.6875687568756875
Precision: 0.3597733711048159
Recall: 0.6864864864864865
F1 Score: 0.47211895910780666
AUC ROC: 0.6871658951769449


**Threshold**

С помощью цикла, посмотрим на значение F1-меры при разном пороге у разных моделей. 

In [40]:
threshold_result = pd.DataFrame(columns = ["F1","AUC ROC"])

In [41]:
best_model = None
best_result = 0
best_threshold = 0
for threshold in np.arange(0.1, 0.9, 0.01):
    model = LogisticRegression(random_state=RANDOM_STATE)
    model.fit(features_train,target_train)
    predicted = (model.predict_proba(features_valid)[:,1] >= threshold).astype(bool)
    result = f1_score(target_valid,predicted)
    if result > best_result:
        best_model = model
        best_result = result
        best_threshold = threshold
    
print("F1 лучшей модели:", best_result)
print("Порог лучшей модели:", best_threshold)
probabilities_one_valid = best_model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))

threshold_result.loc['LR'] = pd.Series({'F1':best_result, 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

F1 лучшей модели: 0.47446457990115326
Порог лучшей модели: 0.16999999999999998
AUC ROC: 0.7455129162311482


In [42]:
best_model = None
best_result = 0
best_threshold = 0
for threshold in np.arange(0.1, 0.9, 0.01):
    model = RandomForestClassifier(random_state=RANDOM_STATE)
    model.fit(features_train,target_train)
    predicted = (model.predict_proba(features_valid)[:,1] >= threshold).astype(bool)
    result = f1_score(target_valid,predicted)
    if result > best_result:
        best_model = model
        best_result = result
        best_threshold = threshold
    
print("F1 лучшей модели:", best_result)
print("Порог лучшей модели:", best_threshold)
probabilities_one_valid = best_model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))

threshold_result.loc['RFC'] = pd.Series({'F1':best_result, 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

F1 лучшей модели: 0.5907859078590786
Порог лучшей модели: 0.34999999999999987
AUC ROC: 0.8396819471405107


In [43]:
best_model = None
best_result = 0
best_threshold = 0
for threshold in np.arange(0.1, 0.9, 0.01):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE)
    model.fit(features_train,target_train)
    predicted = (model.predict_proba(features_valid)[:,1] >= threshold).astype(bool)
    result = f1_score(target_valid,predicted)
    if result > best_result:
        best_model = model
        best_result = result
        best_threshold = threshold
    
print("F1 лучшей модели:", best_result)
print("Порог лучшей модели:", best_threshold)
probabilities_one_valid = best_model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))

threshold_result.loc['DTC'] = pd.Series({'F1':best_result, 'AUC ROC':roc_auc_score(target_valid,probabilities_one_valid)})

F1 лучшей модели: 0.46842105263157896
Порог лучшей модели: 0.1
AUC ROC: 0.6673361206510378


**Результаты**

In [44]:
print("Original:")
display(original_result)
print("Upsampling:")
display(upsampling_result)
print("Weight class:")
display(weight_class_result)
print("Downsampling:")
display(downsampling_result)
print("Threshold:")
display(threshold_result)

Original:


,F1,AUC ROC
LR,0.247934,0.745513
RFC,0.552189,0.839682
DTC,0.468421,0.667336


Upsampling:


,F1,AUC ROC
LR,0.477099,0.749649
RFC,0.577508,0.841343
DTC,0.456044,0.658026


Weight class:


,F1,AUC ROC
LR,0.471954,0.749686
RFC,0.536082,0.845005
DTC,0.459893,0.661300


Downsampling:


,F1,AUC ROC
LR,0.466418,0.749433
RFC,0.561983,0.839510
DTC,0.472119,0.687166


Threshold:


,F1,AUC ROC
LR,0.474465,0.745513
RFC,0.590786,0.839682
DTC,0.468421,0.667336


Сложно выявить лучший вариант балансировки, так как в зависимости от модели, метод балансировки как улучшил, так и ухудшил результат. Однако, в общем, результаты лучше, чем без балансировки, в особенности для LogisticRegression.

Из результатов балансировки можно сделать вывод, что лучшей моделью для данной задачи является RandomForestClassifier, среди всех вариантов балансировки были достигнуты лучшие значения F1-меры и значения AUC ROC в сравнее с остальными моделями.

Лучшей моделью оказалось модель RandomForestClassifier с порогом в 0.35, значение F1-меры составило 0.59, а AUC ROC составило 0.84. Для данной модели будут подобранны гиперпараметры для оптимального результата:

In [45]:
best_gini_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 101, 5):
    for depth in range (1, 51, 5):
        model = RandomForestClassifier(random_state=RANDOM_STATE,n_estimators=est, max_depth=depth,criterion="gini")
        model.fit(features_train,target_train)
        predicted_valid = (model.predict_proba(features_valid)[:,1] >= 0.35).astype(bool)
        result = f1_score(target_valid,predicted_valid)
        if result > best_result:
            best_gini_model = model
            best_result = result
            best_depth = depth
            best_est = est
            
print("Глубина лучшей модели:", best_depth)
print("Число деревьев лучшей модели:", best_est)
print("F1 лучшей модели:", best_result)
probabilities_one_valid = best_gini_model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))

Глубина лучшей модели: 21
Число деревьев лучшей модели: 71
F1 лучшей модели: 0.6162162162162163
AUC ROC: 0.836352097954308


In [46]:
best_entropy_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 101, 5):
    for depth in range (1, 51, 5):
        model = RandomForestClassifier(random_state=RANDOM_STATE,n_estimators=est, max_depth=depth,criterion="entropy")
        model.fit(features_train,target_train)
        predicted_valid = (model.predict_proba(features_valid)[:,1] >= 0.35).astype(bool)
        result = f1_score(target_valid,predicted_valid)
        if result > best_result:
            best_entropy_model = model
            best_result = result
            best_depth = depth
            best_est = est
            
print("Глубина лучшей модели:", best_depth)
print("Число деревьев лучшей модели:", best_est)
print("F1 лучшей модели:", best_result)
probabilities_one_valid = best_entropy_model.predict_proba(features_valid)[:,1]
print("AUC ROC:",roc_auc_score(target_valid,probabilities_one_valid))

Глубина лучшей модели: 16
Число деревьев лучшей модели: 51
F1 лучшей модели: 0.6149584487534626
AUC ROC: 0.8440794385545768


Лучший моделью, по результатам подбора гиперпараметров, оказалось модель RandomForestClassifier со знаением criterion="gini", глубиной 21 и числом деревьев 71.

Данная модель достигла F1-меры 0.616 и AUR ROC 0.836. Далее, модель будет проверена на тестовой выборке. 

## Тестирование модели

Теперь проверим модель на тестовой выборки, сможет ли она достигнуть значение F1 > 0.59?

In [47]:
test_results = (best_gini_model.predict_proba(features_test)[:,1] >= 0.35).astype(bool)

print("Accurasy:", model.score(features_test,target_test))
print("Precision:", precision_score(target_test,test_results))
print("Recall:", recall_score(target_test,test_results))
print("F1 Score:", f1_score(target_test,test_results))
probabilities_one_valid = model.predict_proba(features_test)[:,1]
print("AUC ROC:",roc_auc_score(target_test,probabilities_one_valid))

Accurasy: 0.8769230769230769
Precision: 0.6318407960199005
Recall: 0.6827956989247311
F1 Score: 0.6563307493540051
AUC ROC: 0.8689776035169012


Результат на тестовой выборки - успех. Модель достигла 0.656 F1-меры и 0.869 AUC ROC.

## Вывод

Из проделанной работы, было выявлено, что лучшей моделью для предсказания тарифа является случайный лес с глубинной 11 и числе деревьев 21, и значением criterion - "gini". В данном проекте модель располагается под переменной "best_gini_model". 

Лучшая модель достигла значение F1-меры - 0.656 и значение AUC ROC - 0.869 по проверки на тестовой выборке. Следственно, модель выполняет поставленную цель в F1 > 0.59 и говорит о возможности модели предсказать уход клиента из банка с хорошей точностью.   